# Covid19 Prediction
Construct a nueral network for predicting covid-19 test result.

## Data Preparation and Initialization

### Import Packages

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from warmup_scheduler import GradualWarmupScheduler
import swats

import numpy as np
import csv
import os

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

### Initialization

In [ ]:
myseed = 73  # Set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

### Helper functions

In [ ]:
def get_device():
    ''' Get device (if GPU is available, use GPU) '''
    return 'cuda' if torch.cuda.is_available() else 'cpu'

def plot_learning_curve(loss_record, title=''):
    ''' Plot learning curve of your DNN (train & dev loss) '''
    total_steps = len(loss_record['train'])
    x_1 = range(total_steps)
    x_2 = x_1[::len(loss_record['train']) // len(loss_record['dev'])]
    figure(figsize=(6, 4))
    plt.plot(x_1, loss_record['train'], c='tab:red', label='train')
    plt.plot(x_2, loss_record['dev'], c='tab:cyan', label='dev')
    plt.ylim(0.0, 5.)
    plt.xlabel('Training steps')
    plt.ylabel('MSE loss')
    plt.title('Learning curve of {}'.format(title))
    plt.legend()
    plt.show()


def plot_pred(dv_set, model, device, lim=35., preds=None, targets=None):
    ''' Plot prediction of your DNN '''
    if preds is None or targets is None:
        model.eval()
        preds, targets = [], []
        for x, y in dv_set:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                preds.append(pred.detach().cpu())
                targets.append(y.detach().cpu())
        preds = torch.cat(preds, dim=0).numpy()
        targets = torch.cat(targets, dim=0).numpy()

    figure(figsize=(5, 5))
    plt.scatter(targets, preds, c='r', alpha=0.5)
    plt.plot([-0.2, lim], [-0.2, lim], c='b')
    plt.xlim(-0.2, lim)
    plt.ylim(-0.2, lim)
    plt.xlabel('ground truth value')
    plt.ylabel('predicted value')
    plt.title('Ground Truth v.s. Prediction')
    plt.show()

### Dataset

In [ ]:
class COVID19Dataset(Dataset):
    ''' Dataset for loading and preprocessing the COVID19 dataset '''
    def __init__(self, path, mode='train', target_only=False, fold=0, sample_n=10):
        self.mode = mode

        # Read data into numpy arrays
        with open(path, 'r') as fp:
            data = list(csv.reader(fp))
            data = np.array(data[1:])[:, 1:].astype(float)
        
        feats = list(range(40)) +[40, 41, 42, 43, 57,
                                  58, 59, 60, 61, 75,
                                  76, 77, 78, 79,   ]

        if mode == 'test':
            data = data[:, feats]
            self.data = torch.FloatTensor(data)
        else:
            target = data[:, -1]
            data = data[:, feats]
            
            # Splitting training data into train & dev sets
            if mode == 'train':
                indices = [i for i in range(len(data)) if i % sample_n != fold ]
            elif mode == 'dev':
                indices = [i for i in range(len(data)) if i % sample_n == fold ]
            
            # Convert data into PyTorch tensors
            self.data = torch.FloatTensor(data[indices])
            self.target = torch.FloatTensor(target[indices])

        self.dim = self.data.shape[1]

        print('Finished reading the {} set of COVID19 Dataset ({} samples found, each dim = {})'
              .format(mode, len(self.data), self.dim))

    def __getitem__(self, index):
        # Returns one sample at a time
        if self.mode in ['train', 'dev']:
            # For training
            return self.data[index], self.target[index]
        else:
            # For testing (no target)
            return self.data[index]

    def __len__(self):
        # Returns the size of the dataset
        return len(self.data)

### Dataloader

In [ ]:
def prep_dataloader(path, mode, batch_size, n_jobs=0, target_only=True, fold=0, sample_n=10):
    ''' Generates a dataset, then is put into a dataloader. '''
    dataset = COVID19Dataset(path, mode=mode, target_only=target_only, fold=fold, sample_n=sample_n)  # Construct dataset
    dataloader = DataLoader(
        dataset, batch_size,
        shuffle=(mode == 'train'), drop_last=False,
        num_workers=n_jobs, pin_memory=True) # Construct dataloader
    return dataloader

## Neural Network

### Neural Network Construction

In [ ]:
class NeuralNet(nn.Module):
    ''' A simple fully-connected deep neural network '''
    def __init__(self, input_dim):
        super(NeuralNet, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

        # Mean squared error loss
        self.criterion = nn.MSELoss(reduction='mean')

    def forward(self, x):
        ''' Given input of size (batch_size x input_dim), compute output of the network '''
        return self.net(x).squeeze(1)

    def cal_loss(self, pred, target):
        ''' Calculate loss '''
        return self.criterion(pred, target)


### Model Evaluation Function

In [ ]:
def dev(dv_set, model, device):
    model.eval()                                # set model to evalutation mode
    total_loss = 0
    for x, y in dv_set:                         # iterate through the dataloader
        x, y = x.to(device), y.to(device)       # move data to device (cpu/cuda)
        with torch.no_grad():                   # disable gradient calculation
            pred = model(x)                     # forward pass (compute output)
            rmse_loss = torch.sqrt(model.cal_loss(pred, y))  # compute loss
            total_loss += rmse_loss.detach().cpu().item() * len(x)  # accumulate loss
    total_loss = total_loss / len(dv_set.dataset)              # compute averaged loss

    return total_loss

### Model Training Function

In [ ]:
def train(tr_set, dv_set, model, config, device):
    ''' DNN training '''
    n_epochs = config['n_epochs']  # Maximum number of epochs

    # Setup optimizer
    optimizer = swats.SWATS(model.parameters(), **config['optim_hparas'])
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=3000)
    scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=1, total_epoch=5, after_scheduler=scheduler)

    min_rmse = 1000.
    loss_record = {'train': [], 'dev': []}      # for recording training loss
    early_stop_cnt = 0
    epoch = 0
    while epoch < n_epochs:
        model.train()                           # set model to training mode
        scheduler_warmup.step(epoch)
        for x, y in tr_set:                     # iterate through the dataloader
            optimizer.zero_grad()               # set gradient to zero
            x, y = x.to(device), y.to(device)   # move data to device (cpu/cuda)
            pred = model(x)                     # forward pass (compute output)
            rmse_loss = torch.sqrt(model.cal_loss(pred, y))  # compute loss
            rmse_loss.backward()                 # compute gradient (backpropagation)
            optimizer.step()                    # update model with optimizer
            scheduler.step()
            train_loss = rmse_loss.detach().cpu().item()
            loss_record['train'].append(train_loss)

        # After each epoch, test your model on the validation (development) set.
        dev_rmse = dev(dv_set, model, device)
        if dev_rmse < min_rmse:
            # Save model if the model improved
            min_rmse = dev_rmse
            print('Saving model (epoch = {:4d}, train_loss = {:.4f}, loss = {:.4f})'
                .format(epoch + 1, train_loss, min_rmse))
            torch.save(model.state_dict(), config['save_path'])  # Save model to specified path
            early_stop_cnt = 0
        else:
            early_stop_cnt += 1

        epoch += 1
        loss_record['dev'].append(dev_rmse)
        if early_stop_cnt > config['early_stop']:
            # Stop training if your model stops improving for "config['early_stop']" epochs.
            break

    print('Finished training after {} epochs'.format(epoch))
    return min_rmse, loss_record

### Testing

In [ ]:
def test(tt_set, model, device):
    model.eval()                                # set model to evalutation mode
    preds = []
    for x in tt_set:                            # iterate through the dataloader
        x = x.to(device)                        # move data to device (cpu/cuda)
        with torch.no_grad():                   # disable gradient calculation
            pred = model(x)                     # forward pass (compute output)
            preds.append(pred.detach().cpu())   # collect prediction
    preds = torch.cat(preds, dim=0).numpy()     # concatenate all predictions and convert to a numpy array
    return preds

## Model Training

### Hyperparameters

In [ ]:
device = get_device()                 # Get the current available device ('cpu' or 'cuda')
os.makedirs('models', exist_ok=True)  # The trained model will be saved to ./models/
target_only = True

# Path to the CSV file
tr_path = "./covid.train.csv"
tt_path = "./covid.test.csv"

config = {
    'n_epochs': 3000,                # maximum number of epochs
    'batch_size': 50,               # mini-batch size for dataloader
    'optimizer': 'Adam',              # optimization algorithm
    'optim_hparas': {                # hyper-parameters for the optimizer
        'lr': 0.003,                 # learning rate of SGD
        # 'momentum': 0.9,              # momentum for SGD
        'weight_decay': 0.00022220131864630633
    },
    'early_stop': 500,               # early stopping epochs
    'save_path': 'models/model.pth'\
}

# Cross validation
sample_time = 10
fold = 0

### Saving predictions function

In [ ]:
def save_pred(preds, file):
    ''' Save predictions to specified file '''
    print('Saving results to {}'.format(file))
    with open(file, 'w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['id', 'tested_positive'])
        for i, p in enumerate(preds):
            writer.writerow([i, p])

### Model Training

In [ ]:
for time in range(sample_time):
    print('times: {:4d} /{:4d}'.format(time + 1, sample_time))
    """Load data and model"""
    tr_set = prep_dataloader(tr_path, 'train', config['batch_size'], target_only=target_only, fold=fold, sample_n=sample_time)
    dv_set = prep_dataloader(tr_path, 'dev', config['batch_size'], target_only=target_only, fold=fold, sample_n=sample_time)
    tt_set = prep_dataloader(tt_path, 'test', config['batch_size'], target_only=target_only, fold=fold, sample_n=sample_time)

    model = NeuralNet(tr_set.dataset.dim).to(device)  # Construct model and move to device

    """Start Training"""
    model_loss, model_loss_record = train(tr_set, dv_set, model, config, device)

    plot_learning_curve(model_loss_record, title='deep model')

    del model
    model = NeuralNet(tr_set.dataset.dim).to(device)
    ckpt = torch.load(config['save_path'], map_location='cpu')  # Load the best model
    model.load_state_dict(ckpt)
    plot_pred(dv_set, model, device)  # Show prediction on the validation set

    """
    Testing
    """
    if time == 0:
        avg_preds = test(tt_set, model, device)  # predict COVID-19 cases
        avg_loss = model_loss
    else:
        preds = test(tt_set, model, device)
        avg_preds += preds
        avg_loss += model_loss

    if time == (sample_time - 1):
        avg_preds /= sample_time
        avg_loss /= sample_time
        print('average loss: {:.4f}',format(avg_loss))
        save_pred(avg_preds, 'pred.csv')         # save prediction file to pred.csv

    fold += 1